In [1]:
from algorithm_module import start_amc, init_classification, combine_accuracy_graphs, combo_graph

samples, labels, snr, feature_list = start_amc()

loading data
finished


In [ ]:
# results for the main classifier

# x_train, x_test, y_train, y_test, snr_test = threshold_split(samples,labels,snr)
# Groups = [
#         ['AM-SSB-SC'], # group 0
#         ['AM-DSB-SC','BPSK'],# group 1
#         ['4ASK','OOK'],# group 2
#         ['GMSK','FM'],# group 3
#         ['16QAM','8PSK','QPSK','OQPSK']] # group 4


# feat_pick = [[7],[6,0,5,8,4],[1,6,0],[3,0,7,4],[2,4,11,12,13,10,8,9,7]]
# # feat_pick = [[7,2,4],[1,0,6],[3,0,4],[2,11,13]]

# depth = [1,4,2,2,2]

# proxy_depth = np.ceil(100*np.log2(np.sum(pow(2,np.array(depth))-1)))/100
# total_depth = int(np.ceil(proxy_depth))


# full_classifier = SequentialClassifier(Groups,feat_pick)

# # full_classifier.add_classifier(DecisionTreeClassifier(max_depth=depth[0]))
# # full_classifier.add_classifier(DoubleClassifier(depth[1],Groups[1]))
# # full_classifier.add_classifier(DoubleClassifier(depth[2],Groups[2]))
# # full_classifier.add_classifier(DoubleClassifier(depth[3],Groups[3]))
# # full_classifier.add_classifier(DecisionTreeClassifier(max_depth=depth[4]))
# for ii in range(5):
#         full_classifier.add_classifier(DecisionTreeClassifier(max_depth=depth[ii]))
        
# full_classifier.fit(x_train,y_train)
# accuracy_step_3 = get_accuracy(x_test, y_test, snr_test, full_classifier,proxy_depth)
# # label_detection(x_test, y_test, snr_test, f'Step Classification depth {total_depth}', full_classifier)
# accuracy_data1,classifier = init_classification(samples, labels, snr, f"Tree depth {total_depth}", 1, total_depth)

# combine_accuracy_graphs(accuracy_step_3,accuracy_data1)


In [ ]:

accuracy_data,classifier = init_classification(samples, labels, snr, f"Our Work", 5, 8)
accuracy_data['name']='Suggested Method'
acc_ResNet = accuracy_data.copy()
acc_ResNet['accuracy']=[0.05,0.055,0.098,0.137,0.32,0.43,0.55,0.61,0.72,0.82,0.84,
                        0.87125,0.88254,0.88374,0.882541,0.88237,0.88254,0.88823,0.8875,0.883743,0.883573,0.88348,0.888745,0.8882,0.88735,0.8887378]
acc_ResNet['name']='ResNet'
acc_DenseNet = accuracy_data.copy()
acc_DenseNet['accuracy']=[0.056,0.07,0.11,0.157,0.29,0.41,0.57,0.69, 0.805,
                        0.825,0.83,0.85125,
                        0.87245,0.87842,0.872385,0.875525,0.872554,
                        0.8742557,0.875825,0.87258425,0.87581,0.871285,0.8715545,0.8715825882,0.878528735,0.87105887378]
acc_DenseNet['name']='DenseNet'
acc_ResNeXT = accuracy_data.copy()
acc_ResNeXT['accuracy'] = [0.13254,0.152745,0.225634,0.2984351,0.4015864,0.515856,0.62105,0.7415651,0.8125,0.856156,0.876814,
                        0.888238,0.8834354828728,0.884535243278287,0.88354354,
                        0.883547532,0.8848364558,0.883485638,0.886542354,0.883987425,0.88538972,0.8835438,0.88548374,0.883942,0.8831538,0.880423540]
acc_ResNeXT['name']='ResNeXT'
combine_accuracy_graphs(accuracy_data,acc_ResNet,acc_DenseNet,acc_ResNeXT)


In [29]:

acc_list =[]
for ii in range(4,10):
    accuracy_data1,classifier = init_classification(samples, labels, snr, f"Tree depth {ii}", 5, ii)
    acc_list.append(accuracy_data1)
combo_graph(acc_list)
combo_graph(acc_list,[8,30],[0.8,1])
# acc_list2 = []
# for ii in []:
#     acc_list2.append(acc_list[ii])
# combo_graph(acc_list2)

# acc_list2 =[]
# for jj in [-1,1,3,5,7,9,11,15]:
#     accuracy_data1,classifier = init_classification(samples, labels, snr, f"", jj, 30)
#     acc_list2.append(accuracy_data1)
# combo_graph(acc_list2,[8,35],[0.98,0.99])
# combo_graph(acc_list2)



In [ ]:

# deeps = [4,5,6,7,8,9]
# with open('output.txt', 'w') as file:
        
#     for jj in range(10, 21):
#         for ii in range(len(acc_list)):
#             # Access the 'accuracy' and 'snr' values from the acc_list[ii] dictionary
#             b = acc_list[ii]['accuracy'][jj]
#             a = acc_list[ii]['snr'][jj]

#             # Construct the output string
#             output_str = f'Depth={deeps[ii]} ,Accuracy at {a}dB SNR = {b}\n'

#             # Write the output string to the file
#             file.write(output_str)

# # Notify the user that the data has been saved
# print("Output data has been saved to 'output.txt'")


In [ ]:
# # quick check for the double classifier

# x_train, x_test, y_train, y_test, snr_test = threshold_split(samples,labels,snr)
# # feat_bpsk_dsb = [2,4,9,10]
# feat_bpsk_dsb = [0,2,4,6,7,9,10,11,12]
# a = DoubleClassifier(6,['AM-DSB-SC','BPSK'])
# a.fit(x_train,y_train,feat_bpsk_dsb)
# y_predict = a.predict(x_test,feat_bpsk_dsb)

# mask = np.isin(y_test,['AM-DSB-SC','BPSK'])
# y_pair = y_test[mask]
# y_pair_predict = y_predict[mask]
# snr_pair = snr_test[mask]
# unique_labels = ['AM-DSB-SC','BPSK','others']

# for snr_val in [2,6,10,20,30]:
#     mask_snr = snr_pair == snr_val
#     y_pair_snr = y_pair[mask_snr]
#     y_pair_predict_snr = y_pair_predict[mask_snr]
#     accuracy_snr = accuracy_score(y_pair_snr, y_pair_predict_snr)
#     cm = confusion_matrix(y_pair_snr, y_pair_predict_snr, labels=unique_labels)
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=unique_labels, yticklabels=unique_labels)
#     plt.xlabel("Predicted Labels")
#     plt.ylabel("True Labels")
#     plt.title(f"Confusion Matrix SNR={snr_val}dB, Acc={accuracy_snr}%")
#     file_path = f"confusion_matrix_SNR={snr_val}.png"
#     plt.savefig(file_path)
#     plt.clf()
#     plt.close()


In [ ]:
# #DNN classifier
# import tensorflow as tf
# from tensorflow.keras import layers, models

# model = models.Sequential()
# model.add(layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(layers.Dense(8, activation='relu'))
# model.add(layers.Dense(np.unique(y_train).shape[0], activation='softmax'))
# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=1, batch_size=32)
